<a href="https://colab.research.google.com/github/noushadkhan01/Projects/blob/master/Deep-Learning/ANN/Classification/ANN_model_for_classification_on_Chunk_dataset_for_bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
start = time.time()
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('https://bit.ly/2QMuIPs')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.shape

(10000, 14)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


## check for null values

In [6]:
df.isnull().sum().sort_values(ascending = False)[:2]

Exited             0
EstimatedSalary    0
dtype: int64

**^^as u can see there is no null value**

In [7]:
df.skew()

RowNumber          0.000000
CustomerId         0.001149
CreditScore       -0.071607
Age                1.011320
Tenure             0.010991
Balance           -0.141109
NumOfProducts      0.745568
HasCrCard         -0.901812
IsActiveMember    -0.060437
EstimatedSalary    0.002085
Exited             1.471611
dtype: float64

In [8]:
np.log1p(df.Age).skew()

0.20336041516983316

## import important dependencies

In [0]:
import tensorflow as tf
from tensorflow import keras

In [10]:
df.head(1)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [11]:
df = df.iloc[:, 3:]
df.head(1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [12]:
df = pd.get_dummies(df, drop_first = True)
df.head(1)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.0,1,1,1,101348.88,1,0,0,0


In [13]:
X = df.drop('Exited', 1)
y = df.Exited
X.shape, y.shape

((10000, 11), (10000,))

## StandardScaler

We need to scale out data before applying to neural network becuase of speed

In [0]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7000, 11), (3000, 11), (7000,), (3000,))

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
sc = StandardScaler()

In [18]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train[:1], X_test[:1]

(array([[-0.09792126, -0.55759842, -1.03635146,  1.13249447,  0.81039385,
          0.64198477,  0.96683535, -0.76862426, -0.56987189, -0.5731713 ,
          0.92295821]]),
 array([[-0.55032881, -0.36597914,  1.0436811 ,  0.89172052, -0.92971564,
          0.64198477,  0.96683535,  1.61245299,  1.75478035, -0.5731713 ,
         -1.08347268]]))

In [19]:
y_train = y_train.values
y_test = y_test.values
y_train[:2], y_test[:2]

(array([1, 0]), array([0, 1]))

# Make ANN

In [0]:
from tensorflow import keras

## import Sequential from keras
**it is required to initialize our neural network model**

### Step 1:- create ann model by sequential

In [0]:
#1 first initialize our keras model by Sequential
model = keras.models.Sequential()
model

### Step 2:- Add dense layer

if a data is linenarly separable then their is no need to add hidden layer or don't even need a neural networks by convention

**Tip:-- ** no. of nodes in hidden layers is the average of no. of nodes in input layers and no. of node in output layer

In [0]:
ninput = 11 #no of independent variables
noutput = 1 #output is either 1 or either 0 so no of hidden layers in output is 1
nhidden = (ninput + noutput) / 2
nhidden

6.0

In [0]:
model.add(keras.layers.Dense(units = 6, activation = tf.nn.relu, input_dim = 11))

W0808 17:17:57.034063 140200105752448 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model

### add 2nd layer

In [0]:
model.add(keras.layers.Dense(units = 6, activation = tf.nn.relu))
model

### Step 3:- add output layer

**there are two types of activation function for output of categorical varibles **
- 1**softmax**, which is used when dependent varible have more than two categorical variables
- 2 **sigmoid** , which is used when dependent varable have 2 categorical variables

In [0]:
model.add(keras.layers.Dense(units = 1, activation = tf.nn.sigmoid))

In [0]:
model

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


## Step 4:-- compile the model
apply stochastic gradient descent method

In [0]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model

W0808 17:17:57.259725 140200105752448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


###all step in one one method

In [0]:
def create_model():
  import tensorflow as tf
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(6, activation = tf.nn.relu, input_dim = 11),
    tf.keras.layers.Dense(6, activation = tf.nn.relu),
    tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)])
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  return model

### Step 5:-- fit the model

batch_size = no. of observation afterwhich u want to update the weights

In [0]:
import os
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [0]:
import time
start = time.time()
model.fit(X_train, y_train, batch_size = 10 , epochs = 70,verbose = 0,  class_weight = 'balanced', 
          validation_data = (X_test, y_test), #callbacks = [cp_callback] use callback if you want to save model in checkpoints
          )
end = time.time()
tpu_time = end - start
print(f'tpu run time is {tpu_time}')

Train on 7000 samples, validate on 3000 samples
Epoch 1/70
7000/7000 [==============================] - 1s 200us/sample - loss: 0.5116 - acc: 0.7571 - val_loss: 0.4485 - val_acc: 0.7930
Epoch 2/70
7000/7000 [==============================] - 1s 146us/sample - loss: 0.4311 - acc: 0.7977 - val_loss: 0.4361 - val_acc: 0.7930
Epoch 3/70
7000/7000 [==============================] - 1s 145us/sample - loss: 0.4227 - acc: 0.7977 - val_loss: 0.4290 - val_acc: 0.7930
Epoch 4/70
7000/7000 [==============================] - 1s 146us/sample - loss: 0.4156 - acc: 0.7977 - val_loss: 0.4219 - val_acc: 0.7930
Epoch 5/70
7000/7000 [==============================] - 1s 146us/sample - loss: 0.4092 - acc: 0.8086 - val_loss: 0.4179 - val_acc: 0.8213
Epoch 6/70
7000/7000 [==============================] - 1s 146us/sample - loss: 0.4030 - acc: 0.8264 - val_loss: 0.4110 - val_acc: 0.8213
Epoch 7/70
7000/7000 [==============================] - 1s 149us/sample - loss: 0.3969 - acc: 0.8280 - val_loss: 0.4044 - va

In [0]:
tpu_time

72.03990364074707

### Save model with save method

In [0]:
model.save('my_ann_model_without_dropout.h5')

In [0]:
#!ls {checkpoint_dir} for checkp

###Train score

In [0]:
model.evaluate(X_train, y_train, batch_size = 10)

7000/7000 [==============================] - 0s 63us/sample - loss: 0.3406 - acc: 0.8614


[0.34062693089778934, 0.86142856]

###test score

In [0]:
model.evaluate(X_test, y_test)

3000/3000 [==============================] - 0s 25us/sample - loss: 0.3441 - acc: 0.8587


[0.34413528577486674, 0.85866666]

### Step 7:- make predictions

In [0]:
y_pred = model.predict(X_test)
y_pred[:2]

array([[0.2118898 ],
       [0.34375626]], dtype=float32)

In [0]:
y_pred = y_pred.reshape((y_pred.shape[0]))

In [0]:
y_pred

array([0.2118898 , 0.34375626, 0.14856061, ..., 0.19052774, 0.04095182,
       0.5987141 ], dtype=float32)

In [0]:
y_test

array([0, 1, 0, ..., 0, 0, 1])

In [0]:
y_pred.shape, y_test.shape

((3000,), (3000,))

In [0]:
from sklearn.metrics import confusion_matrix
f_ypred = [1 if i > .5 else 0 for i in y_pred]
confusion_matrix(y_test, f_ypred)

array([[2284,   95],
       [ 329,  292]])

##Cross validation

to use cross validation with keras models we need to import **KerasClassifier **from keras.models

In [0]:
def my_keras_cross_val_score(build_fn):
  from keras.wrappers.scikit_learn import KerasClassifier
  from sklearn.model_selection import cross_val_score
  classifier = KerasClassifier(build_fn = build_fn, batch_size = 10, epochs = 70, verbose = 1)
  import time
  start = time.time()
  scores = cross_val_score(classifier, X_train, y_train, cv = 10, n_jobs = -1, verbose = 1)
  end = time.time()
  tpu_runtime = end - start
  print(f'tpu run time is {tpu_runtime}')
  return scores

In [0]:
my_keras_cross_val_score(create_model)

In [0]:
scores

array([0.85857141, 0.88571429, 0.85285717, 0.86714286, 0.86714286,
       0.84857142, 0.85428572, 0.85714287, 0.86142856, 0.86142856])

In [0]:
scores.mean(), scores.std()

(0.8614285707473754, 0.009856107709771569)

##How to use saved model

In [0]:
model2 = tf.keras.models.load_model('my_ann_model_without_dropout.h5')
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [0]:
test_loss, test_score = model2.evaluate(X_test, y_test)

3000/3000 [==============================] - 0s 28us/sample - loss: 0.3441 - acc: 0.8587


In [0]:
test_loss, test_score

(0.34413528577486674, 0.85866666)

In [0]:
train_loss, test_loss = model2.evaluate(X_train, y_train)
train_loss, test_loss

7000/7000 [==============================] - 0s 24us/sample - loss: 0.3406 - acc: 0.8614


(0.34062693190574644, 0.86142856)

##Dropout

dropout is useful when somehow we overfitted the trainig dataset so we need to dismantle some neurons from the layers 
like .2 dropout means 20 % of neurons are dismantled from the layers after which we have implemented the dropout layer

In [0]:
def create_dropout_model():
  import tensorflow as tf
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(6, activation = tf.nn.relu, input_dim = 11),
    tf.keras.layers.Dropout(rate = .1, seed = 0),
    tf.keras.layers.Dense(6, activation = tf.nn.relu),
    tf.keras.layers.Dropout(rate = .1, seed = 0),
    tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)])
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  return model
model3 = create_dropout_model()
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 6)                 72        
_________________________________________________________________
dropout_2 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 42        
_________________________________________________________________
dropout_3 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [0]:
start = time.time()
model3.fit(X_train, y_train, batch_size = 10, epochs = 70, validation_data = (X_test, y_test), verbose = 0)
end = time.time()
tpu_runtime = end - start
print(f'tpu runtime is {tpu_runtime}')

tpu runtime is 75.71206855773926


In [0]:
model3.evaluate(X_train, y_train)

7000/7000 [==============================] - 0s 26us/sample - loss: 0.3433 - acc: 0.8570


[0.3433252932344164, 0.857]

In [0]:
model3.evaluate(X_test, y_test)

3000/3000 [==============================] - 0s 32us/sample - loss: 0.3471 - acc: 0.8633


[0.34714239565531413, 0.86333334]

**Now it's may be a good model**

In [0]:
model3.save('my_ann_with_dropout.h5')

In [0]:
from google.colab import files
files.download('my_ann_with_dropout.h5')

In [0]:
files.download('my_ann_model_without_dropout.h5')

#Parameter tuning with Scikit_learn's GridSearchCV

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
def create_model_for_grid_search(optimizer, units):
  import tensorflow as tf
  model = tf.keras.models.Sequential([
              tf.keras.layers.Dense(units = units, activation = tf.nn.relu, input_dim = 11),
              tf.keras.layers.Dropout(0.1),
              tf.keras.layers.Dense(units = units, activation = tf.nn.relu),
              tf.keras.layers.Dropout(0.1),
              tf.keras.layers.Dense(units = 1, activation = tf.nn.sigmoid)
              ])
  model.compile(optimizer = optimizer,loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [20]:
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [0]:
classifier = KerasClassifier(build_fn = create_model_for_grid_search)

In [0]:
param_grid = {'units': [6],
              'optimizer': ['adam', 'rmsprop'],
              'batch_size': [10],
              'epochs': [70],
              }

In [0]:
grid = GridSearchCV(classifier, param_grid, scoring = 'accuracy', n_jobs = -1, cv = 3, verbose = 10)

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
sc = StandardScaler()

In [38]:
X[:2]

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0


In [39]:
y[:2]

0    1
1    0
Name: Exited, dtype: int64

In [40]:
X_sc = sc.fit_transform(X)
X_sc[:2]

array([[-0.32622142,  0.29351742, -1.04175968, -1.22584767, -0.91158349,
         0.64609167,  0.97024255,  0.02188649, -0.57873591, -0.57380915,
        -1.09598752],
       [-0.44003595,  0.19816383, -1.38753759,  0.11735002, -0.91158349,
        -1.54776799,  0.97024255,  0.21653375, -0.57873591,  1.74273971,
        -1.09598752]])

In [0]:
param_grid = {'units': [4, 6, 10],
              'optimizer': ['adam', 'rmsprop'],
              'batch_size': [5, 10],
              'epochs': [50, 70],
              }

In [0]:
grid = GridSearchCV(estimator = classifier, param_grid = param_grid, scoring = 'accuracy', n_jobs = -1, cv = 2, verbose = 10)

In [55]:
import time
start = time.time()
grid.fit(X_sc, y)
end = time.time()
tpu_runtime = end - start
print(f'tpu running time of this script is {tpu_runtime}')

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 16.2min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 19.7min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 23.0min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 27.9min finished


Epoch 1/50
10000/10000 [==============================] - 2s 167us/sample - loss: 0.5078 - acc: 0.7690
Epoch 2/50
10000/10000 [==============================] - 2s 154us/sample - loss: 0.4488 - acc: 0.7986
Epoch 3/50
10000/10000 [==============================] - 2s 155us/sample - loss: 0.4312 - acc: 0.8152
Epoch 4/50
10000/10000 [==============================] - 2s 155us/sample - loss: 0.4077 - acc: 0.8290
Epoch 5/50
10000/10000 [==============================] - 2s 154us/sample - loss: 0.3906 - acc: 0.8432
Epoch 6/50
10000/10000 [==============================] - 2s 152us/sample - loss: 0.3748 - acc: 0.8479
Epoch 7/50
10000/10000 [==============================] - 2s 152us/sample - loss: 0.3700 - acc: 0.8472
Epoch 8/50
10000/10000 [==============================] - 2s 156us/sample - loss: 0.3636 - acc: 0.8491
Epoch 9/50
10000/10000 [==============================] - 2s 154us/sample - loss: 0.3617 - acc: 0.8493
Epoch 10/50
10000/10000 [==============================] - 2s 155us/sampl

In [56]:
grid.best_params_

{'batch_size': 10, 'epochs': 50, 'optimizer': 'adam', 'units': 10}

In [59]:
grid.best_score_

0.8612

In [0]:
def create_model():
  model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(units = 10, activation = tf.nn.relu, input_dim = 11),
                                    tf.keras.layers.Dropout(rate = 0.1),
                                    tf.keras.layers.Dense(units = 10, activation = tf.nn.relu),
                                    tf.keras.layers.Dropout(rate = 0.1),
                                    tf.keras.layers.Dense(units = 1, activation = tf.nn.sigmoid)
  ])
  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [82]:
model = create_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 10)                120       
_________________________________________________________________
dropout_6 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_7 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [66]:
model.fit(X_train, y_train,batch_size = 10, epochs = 50, class_weight = 'balanced', validation_data = (X_test, y_test))

Train on 7000 samples, validate on 3000 samples
Epoch 1/50
7000/7000 [==============================] - 2s 249us/sample - loss: 0.5132 - acc: 0.7723 - val_loss: 0.4467 - val_acc: 0.8097
Epoch 2/50
7000/7000 [==============================] - 1s 197us/sample - loss: 0.4527 - acc: 0.8077 - val_loss: 0.4306 - val_acc: 0.8140
Epoch 3/50
7000/7000 [==============================] - 1s 197us/sample - loss: 0.4327 - acc: 0.8159 - val_loss: 0.4131 - val_acc: 0.8273
Epoch 4/50
7000/7000 [==============================] - 1s 199us/sample - loss: 0.4168 - acc: 0.8214 - val_loss: 0.3954 - val_acc: 0.8433
Epoch 5/50
7000/7000 [==============================] - 1s 196us/sample - loss: 0.3998 - acc: 0.8349 - val_loss: 0.3761 - val_acc: 0.8520
Epoch 6/50
7000/7000 [==============================] - 1s 204us/sample - loss: 0.3862 - acc: 0.8421 - val_loss: 0.3653 - val_acc: 0.8577
Epoch 7/50
7000/7000 [==============================] - 1s 197us/sample - loss: 0.3825 - acc: 0.8477 - val_loss: 0.3605 - va

In [67]:
model.evaluate(X_train, y_train)

7000/7000 [==============================] - 0s 25us/sample - loss: 0.3313 - acc: 0.8650


[0.33126282599994117, 0.865]

In [68]:
model.evaluate(X_test, y_test)

3000/3000 [==============================] - 0s 27us/sample - loss: 0.3399 - acc: 0.8603


[0.33993459765116374, 0.8603333]

In [69]:
y_pred = model.predict_classes(X_test)
y_pred[:2]

array([[0],
       [0]], dtype=int32)

In [0]:
from sklearn.metrics import confusion_matrix, classification_report

In [71]:
confusion_matrix(y_test, y_pred)

array([[2304,   75],
       [ 344,  277]])

In [74]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92      2379
           1       0.79      0.45      0.57       621

    accuracy                           0.86      3000
   macro avg       0.83      0.71      0.74      3000
weighted avg       0.85      0.86      0.84      3000



#Handle Class imbalance

##1. SMOTE

In [0]:
from imblearn.over_sampling import SMOTE, ADASYN

In [0]:
sm = SMOTE(random_state = 0)

In [78]:
X_smote, y_smote = sm.fit_resample(X_sc, y)
X_smote.shape, y_smote.shape

((15926, 11), (15926,))

In [0]:
model_sm = create_model()

In [0]:
from sklearn.model_selection import train_test_split

In [87]:
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_smote, y_smote, test_size = 0.3, random_state = 0)
X_train_sm.shape,X_test_sm.shape, y_train_sm.shape, y_test_sm.shape

((11148, 11), (4778, 11), (11148,), (4778,))

In [88]:
model_sm.fit(X_train_sm, y_train_sm, epochs = 50, batch_size = 10,  validation_data = (X_test, y_test), verbose = 0)

In [89]:
model_sm.evaluate(X_train_sm, y_train_sm)

11148/11148 [==============================] - 0s 27us/sample - loss: 0.4284 - acc: 0.7973


[0.4283800165810207, 0.79727304]

In [90]:
model_sm.evaluate(X_test_sm, y_test_sm)

4778/4778 [==============================] - 0s 42us/sample - loss: 0.4521 - acc: 0.7821


[0.45206666549113367, 0.7821264]

In [0]:
def matrix(model, X_test, y_test):
  y_pred = model.predict_classes(X_test)
  from sklearn.metrics import confusion_matrix, classification_report
  print(confusion_matrix(y_test, y_pred))
  print(classification_report(y_test, y_pred))

In [92]:
matrix(model_sm, X_test_sm, y_test_sm)

[[1961  423]
 [ 618 1776]]
              precision    recall  f1-score   support

           0       0.76      0.82      0.79      2384
           1       0.81      0.74      0.77      2394

    accuracy                           0.78      4778
   macro avg       0.78      0.78      0.78      4778
weighted avg       0.78      0.78      0.78      4778



##2. ADASYN

In [0]:
ad = ADASYN(random_state = 0)

In [94]:
X_ad, y_ad = ad.fit_resample(X_sc, y)
X_ad.shape, y_ad.shape

((15959, 11), (15959,))

In [95]:
model_ad = create_model()
model_ad.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 10)                120       
_________________________________________________________________
dropout_10 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_11 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 11        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [96]:
X_train, X_test, y_train, y_test = train_test_split(X_ad, y_ad, random_state = 0, test_size = 0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11171, 11), (4788, 11), (11171,), (4788,))

In [97]:
model_ad.fit(X_train, y_train, batch_size = 10, epochs = 50, verbose = 0, validation_data = (X_test, y_test))

In [98]:
matrix(model_ad, X_test, y_test)

[[1674  697]
 [ 483 1934]]
              precision    recall  f1-score   support

           0       0.78      0.71      0.74      2371
           1       0.74      0.80      0.77      2417

    accuracy                           0.75      4788
   macro avg       0.76      0.75      0.75      4788
weighted avg       0.76      0.75      0.75      4788

